In [1]:
import numpy as np

In [2]:
def categorical_crossentropy(t,y):
    return np.mean(-t*np.log(y))

In [3]:
def sigmoid(x):
    return 1/(1+np.exp(-x+0.001))

In [4]:
def softmax(x):
    c = np.max(x,axis=1).reshape(-1,1)
    x = x-c
    return np.exp(x)/np.sum(np.exp(x),axis=1).reshape(-1,1)

In [5]:
import numpy as np
class Relu:
    def __init__(self):
        self.mask = None
    def forward(self,x):
        self.mask = (x <=0)
        out = x.copy()
        out[self.mask] = 0
        return out
    def backward(self,dout):
        dout[self.mask] = 0
        dx = dout
        return dx
        
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self,x):
        out = sigmoid(x)
        self.out = out
        return out
    
    def backward(self,dout):
        dx = ((1-self.out)*self.out)*dout
        return dx

class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None
        self.origin_shape = None
        self.dW = None
        self.db = None
    
    def forward(self,x):
        self.origin_shape = x.shape
        self.x = x
        out = np.dot(x,self.W) + self.b
        return out
    
    def backward(self,dout):
        dx = np.dot(dout,self.W.T)
        self.dW = np.dot(self.x.T,dout)
        self.db = np.sum(dout,axis=0)
        dx = dx.reshape(self.origin_shape)
        return dx

class Loss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
    
    def forward(self,t,y):
        self.y = softmax(y)
        self.t = t
        self.loss = categorical_crossentropy(self.t, self.y)
        return self.loss
    
    def backward(self,dout=1):
        dx = (self.y - self.t)*dout
        return dx

In [6]:
class Layers:
    def __init__(self):
        self.layers = {}
        
    def add(self,x1,x2,activation):
        activation_dict = {
            'sigmoid':Sigmoid,
            'relu':Relu,
            'softmax':Loss
        }
        w = np.random.randn(x1,x2)
        b = np.zeros(x2)
        activation_layer = 'activation'+str((int(len(self.layers)/2+1))) 
        Affine_layer = 'Affine'+str((int(len(self.layers)/2+1)))
        self.layers[Affine_layer] = Affine(w,b)
        self.layers[activation_layer] = activation_dict[activation]()
        
    
    def predict(self,x):
        out = x.copy()
        ind = 1 
        layer_len = len(self.layers)
        for key, layer in self.layers.items():
            if ind < layer_len :
                out = layer.forward(out)
            ind += 1
        return out
            
    
    def loss(self,x,t):
        y = self.predict(x)
        out = list(self.layers.values())[-1].forward(t,y)
        return out
    
    def accuracy(self,x,t):
        y = self.predict(x)
        y = np.argmax(y,axis=1)
        t = np.argmax(t,axis=1)
        self.acc = np.sum(y==t)/t.size
        return self.acc
    
    def gradient(self,x,t):
        self.loss(x,t)
        lr = 1e-4
        dout = 1
        dout = list(self.layers.values())[-1].backward(dout)
        layers = list(self.layers.values())[::-1][1:]
        self.layers_key = list(self.layers.keys())[::-1][1:]
        for layer in layers:
            dout = layer.backward(dout)
        self.grads = {}
        for layer_key in self.layers_key:
            if 'Affine' in layer_key:
                self.grads[layer_key] = [self.layers[layer_key].dW, self.layers[layer_key].db]
        for layer_key in self.layers_key:
            if 'Affine' in layer_key:
                self.layers[layer_key].W -= lr*self.grads[layer_key][0]
                self.layers[layer_key].b -= lr*self.grads[layer_key][1]
        result = self.loss(x,t)       
        return result
    
    def fit(self,x,t,epochs,lr):
        self.lr = lr
        self.history = {}
        accuracy = []
        loss = []
        for epoch in range(epochs):
            self.gradient(x,t)
            loss.append(self.err)
            accuracy.append(self.accuracy(x,t))
            if epoch % 100 == 0:
                print(f'loss : {self.err} === accuracy : {self.accuracy(x,t)}')
        self.history['accuracy'] = accuracy
        self.history['loss'] = loss
                

In [1]:
from tensorflow import keras

c:\Users\admin\anaconda3\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\admin\anaconda3\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\admin\anaconda3\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\admin\anaconda3\envs\keras\lib\site-packages\tensorflow\python\f

In [2]:
import tensorflow as tf


In [3]:
from keras.datasets import mnist
from keras.utils import to_categorical

Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
c:\Users\admin\anaconda3\envs\keras\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


AttributeError: module 'tensorflow._api.v1.compat.v2.compat' has no attribute 'v1'

In [9]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

NameError: name 'mnist' is not defined

In [ ]:
print(X_train.shape)
print(X_test.shape)


In [ ]:
X_train = X_train.reshape(-1, 28*28)

In [ ]:
y_train.shape
np.unique(y_train)


In [ ]:
model = Layers()

In [ ]:
model.add(784, 2048, 'relu')
model.add(2048, 1024, 'relu')
model.add(1024, 512, 'relu')
model.add(512, 10, 'softmax')

In [ ]:
model.predict(X_train).shape